# Big data lab

## import library

In [54]:
import pandas as pd
import pickle
import my_func
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')
import numpy as np
from sklearn.model_selection import KFold 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
from datetime import datetime, timedelta
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()

fe_name_tr = 'churn_train_model_fe.pickle'
b_num_name_tr ='churn_train_model_b_num.pickle'
dpi_name_tr ='churn_train_model_dpi.pickle'

fe_name_te = 'churn_test_model_fe.pickle'
b_num_name_te ='churn_test_model_b_num.pickle'
dpi_name_te ='churn_test_model_dpi.pickle'

file_path = r'D:\BDL-Final\data-from-BDL' + '\\'

# settings for dataframe
pd.set_option('display.max_columns', None)

In [30]:
# Load dataframes
df_train_fe = pd.read_pickle(file_path+fe_name_tr)
df_train_dpi = pd.read_pickle(file_path+dpi_name_tr)
df_train_b_num = pd.read_pickle(file_path+b_num_name_tr)

df_test_fe = pd.read_pickle(file_path+fe_name_te)
df_test_dpi = pd.read_pickle(file_path+dpi_name_te)
df_test_b_num = pd.read_pickle(file_path+b_num_name_te)



In [4]:
# find out how data looks like
# find unique values of applications
#len(df_train_fe.columns)
# df_apli_tr = df_train_dpi['Application'].unique()
# df_apli_te = df_test_dpi['Application'].unique()
#df_exel = df_train_dpi.iloc[0:1000000]
#df_exel.to_excel('dpi.xlsx')

In [9]:
# apl_te= pd.DataFrame(df_apli_te)
# apl_tr= pd.DataFrame(df_apli_tr)

# #len(df_apli_tr)

In [16]:
#df_train_fe

## Classification Metrics

In [3]:

def binary_classification_metrics(y_true_tr, y_pred_tr, y_true_val = None, y_pred_val = None, report = False):
    print ("{:<15} {:<10} {:<10} {:<10}".format('Metrics','Train','Test','\u0394'))
    metrics_dict = {}

    metrics_dict['roc_auc'] = np.round(roc_auc_score(y_true_tr, y_pred_tr),4)
    metrics_dict['accuracy'] = np.round(accuracy_score(y_true_tr, y_pred_tr),4)
    metrics_dict['precision'] = np.round(precision_score(y_true_tr, y_pred_tr),4)
    metrics_dict['recall'] = np.round(recall_score(y_true_tr, y_pred_tr),4)
    metrics_dict['f1_score'] = np.round(f1_score(y_true_tr, y_pred_tr),4)
    
    if y_true_val is not None:
        metrics_dict_test = {}
        metrics_dict_test['roc_auc'] = np.round(roc_auc_score(y_true_val, y_pred_val),4)
        metrics_dict_test['accuracy'] = np.round(accuracy_score(y_true_val, y_pred_val),4)
        metrics_dict_test['precision'] = np.round(precision_score(y_true_val, y_pred_val),4)
        metrics_dict_test['recall'] = np.round(recall_score(y_true_val, y_pred_val),4)
        metrics_dict_test['f1_score'] = np.round(f1_score(y_true_val, y_pred_val),4)
        # print(classification_report(y_true_val, y_pred_val))
    
        for metrics, value in metrics_dict.items():
            value_test = metrics_dict_test[metrics]
            diff = np.round(metrics_dict_test[metrics] - value, 4)
            print ("{:<15} {:<10} {:<10} {:<10}".format(metrics, value, value_test, diff))
    else:
        for metrics, value in metrics_dict.items():
            print ("{:<15} {:<10}".format(metrics, value))
    if report:
        print('\n')
        print('Train :')
        print(classification_report(y_true_tr, y_pred_tr))
        if not y_true_val.empty:
            print('Test :')
            print(classification_report(y_true_val, y_pred_val))

## First look

In [1]:
#df_train_fe.describe()

In [4]:
all_t = len(df_train_fe['target'])
tgt = df_train_fe['target'].sum()
res =tgt/all_t
res

0.06389333333333333

In [13]:
# перевірка на категоріальні значення
cat_info = my_func.categorical_col_info(df_train_fe)

In [14]:
cat_info

,Column,Unique values,Missing values
Total,0,0,0


# Feature Engineering


In [9]:
# перевірка на missing значення
ms_val = my_func.count_missing_values(df_train_fe)
ms_val

,Column,Missing Values,Missing Values (%)
709,bs_succ_rate,150000,100.000000
708,bs_drop_call_rate,150000,100.000000
645,device_has_gprs,150000,100.000000
716,bs_of_succ_but_drop_m1,150000,100.000000
715,bs_of_unsucc_attemp_equip_m1,150000,100.000000
...,...,...,...
632,loc_market_share,880,0.586667
631,loc_cnt_events,880,0.586667
630,loc_is_obl_center,880,0.586667
606,lt,673,0.448667


## Working with missing values

In [23]:


# Filter the DataFrame to select columns with 100% missing values
df_missing_100 = ms_val[ms_val['Missing Values (%)'] == 100.0]

# Print the resulting DataFrame
df_missing_100.describe()


,Missing Values,Missing Values (%)
count,11.0,11.0
mean,150000.0,100.0
std,0.0,0.0
min,150000.0,100.0
25%,150000.0,100.0
50%,150000.0,100.0
75%,150000.0,100.0
max,150000.0,100.0


In [ ]:
# preparing dataset for base modeling

In [5]:
# prepare dataset for training
def ds_base_prep(df):
    # missing change for -1
    df.fillna(-1, inplace=True)

    # remove abon_id
    return(df)

    

In [6]:
df_tr = ds_base_prep(df_train_fe)

In [7]:
y = df_tr['target']
X = df_tr.drop(['target', 'abon_id'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=777, shuffle=True)


## Train Model

### Base Model

In [12]:
import lightgbm as lgb
def my_trainer_lgb(model, X_train):
    kf = KFold(n_splits=5, shuffle=True, random_state=777)
    callback2 = lgb.log_evaluation(period = 50)
    list_tr_acc = []
    list_val_acc = []
    n=0
    for train_index, val_index in kf.split(X_train):
        n +=1
        # print('Fold№', n)
        X_tr, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
        y_tr, y_val = y_train.iloc[train_index], y_train.iloc[val_index]
        
        model.fit(X_tr, y_tr, eval_set=[(X_tr, y_tr),(X_val, y_val)], eval_names=['Train', 'Val'], eval_metric=('binary_error'), callbacks=[callback2,lgb.early_stopping(20)])
        
        y_tr_pred = model.predict(X_tr)
        y_val_pred = model.predict(X_val)
        
        list_tr_acc.append(accuracy_score(y_tr, y_tr_pred))
        list_val_acc.append(accuracy_score(y_val, y_val_pred))

    print('total Accuracy train= ', np.mean(np.array(list_tr_acc)))
    print('total Accuracy val= ', np.mean(np.array(list_val_acc)))
    return np.mean(np.array(list_val_acc))  - (np.mean(np.array(list_tr_acc)) - np.mean(np.array(list_val_acc)))

In [9]:
from hyperopt import hp, fmin, tpe
import numpy as np
from sklearn.metrics import auc
from sklearn.model_selection import cross_val_score, KFold

In [10]:
space = {
    'learning_rate': hp.choice('learning_rate', np.arange(0.001,0.1, 0.001)),
    'max_depth':    hp.choice("max_depth",  np.arange(2,7,1)),
    'num_leaves': hp.choice('num_leaves', np.arange(10, 128, 10, dtype=int))}

In [14]:
def objective(params):
    param = {   'objective': 'binary',
                'metric': 'auc',
                'num_boost_round': 500,
                'is_unbalance': True}
    clf = lgb.LGBMClassifier(
        n_jobs=-1,
        **param,
        **params
    )  
    score = my_trainer_lgb(clf, X_train)
    print("AUC {:.3f} params {}".format(score, params))
    return -score

In [17]:
best = fmin(fn=objective, 
           space = space,
           algo=tpe.suggest,
           max_evals=50, return_argmin=False)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

c:\Program Files\Python310\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")



Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:                    
[17]	Train's binary_error: 0.0550833	Train's auc: 0.897374	Val's binary_error: 0.0561667	Val's auc: 0.884884
  0%|          | 0/50 [00:08<?, ?trial/s, best loss=?]

c:\Program Files\Python310\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")



[LightGBM] [Warning] num_iterations is set=500, num_boost_round=500 will be ignored. Current value: num_iterations=500
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:                    
[18]	Train's binary_error: 0.0542917	Train's auc: 0.898962	Val's binary_error: 0.0544167	Val's auc: 0.878644
  0%|          | 0/50 [00:19<?, ?trial/s, best loss=?]

...

[LightGBM] [Warning] num_iterations is set=500, num_boost_round=500 will be ignored. Current value: num_iterations=500
Training until validation scores don't improve for 20 rounds                      
Early stopping, best iteration is:                                                
[1]	Train's binary_error: 0.0632812	Train's auc: 0.850298	Val's binary_error: 0.0662917	Val's auc: 0.850388
 98%|█████████▊| 49/50 [38:16<00:47, 47.71s/trial, best loss: -0.9424437500000002]

c:\Program Files\Python310\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")



[LightGBM] [Warning] num_iterations is set=500, num_boost_round=500 will be ignored. Current value: num_iterations=500
Training until validation scores don't improve for 20 rounds                      
Early stopping, best iteration is:                                                
[1]	Train's binary_error: 0.063375	Train's auc: 0.851203	Val's binary_error: 0.0659167	Val's auc: 0.848697
 98%|█████████▊| 49/50 [38:25<00:47, 47.71s/trial, best loss: -0.9424437500000002]

c:\Program Files\Python310\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")



[LightGBM] [Warning] num_iterations is set=500, num_boost_round=500 will be ignored. Current value: num_iterations=500
Training until validation scores don't improve for 20 rounds                      
Early stopping, best iteration is:                                                
[1]	Train's binary_error: 0.0639583	Train's auc: 0.850293	Val's binary_error: 0.0635833	Val's auc: 0.843965
total Accuracy train=                                                             
0.9361166666666666                                                                
total Accuracy val=                                                               
0.9361166666666666                                                                
AUC 0.936 params {'learning_rate': 0.004, 'max_depth': 4, 'num_leaves': 80}       
100%|██████████| 50/50 [38:34<00:00, 46.29s/trial, best loss: -0.9424437500000002]


In [24]:
def save_model(model = None, features = []):
    name = str(model.__class__).split('.')[-1][:-2] + '_' + datetime.today().strftime("%d%m%Y_%H_%M") + '.pickle'    
    if model:
        with open(name, 'wb') as file:
            pickle.dump((model, features), file)
            print('Save', name)

In [26]:
param = {
    'objective': 'binary',
    'metric': 'auc',
    'num_boost_round': 500,
    'is_unbalance': True
}

In [27]:
best_model = lgb.LGBMClassifier(
    n_jobs=-1,
    **param,
    **best
)
save_model(model=best_model, features=X.columns.tolist())

Save LGBMClassifier_24062023_10_47.pickle


In [40]:
best

{'learning_rate': 0.038, 'max_depth': 5, 'num_leaves': 60}

In [ ]:
param = {'objective': 'binary',
        'num_boost_round': 1000,
         'metric': 'auc',
         'is_unbalance': True}
best.update(param)
clf = lgb.LGBMClassifier(**best)

In [ ]:
def model_fit(model, X_tr, y_tr, X_val, y_val):
    callback2 = lgb.log_evaluation(period = 10)
    model.fit(X_tr, y_tr, eval_set=[(X_tr, y_tr),(X_val, y_val)], eval_names=['Train', 'Val'] , early_stopping_rounds=10, eval_metric=['binary_error'], callbacks=[callback2]) 
    iterr = model.best_iteration_
    y_tr_pred = model.predict(X_tr, num_iteration=iterr)
    y_val_pred = model.predict(X_val, num_iteration=iterr)
    print('Accuracy train =', accuracy_score(y_tr, y_tr_pred))
    print('Accuracy val =', accuracy_score(y_val, y_val_pred))
    return model

In [ ]:
clf = model_fit(clf, X, y, X_test, y_test)

## Predict on Test sample


In [45]:
lgb_model = my_func.load_pickle('LGBMClassifier_24062023_10_47.pickle')

In [48]:
y_test = df_test_fe['target']
X_test = df_test_fe.drop(['target', 'abon_id'], axis=1)
X_test.replace(np.inf, -1, inplace=True)
X_test.fillna(-1, inplace=True)

In [53]:
pred_lgb = lgb_model.predict(X_train)
pred_lgb_test = lgb_model.predict_proba(X_test)[:,1]

AttributeError: 'tuple' object has no attribute 'predict'

In [44]:
clf = lgb.LGBMClassifier(**best)

In [ ]:
#binary_classification_metrics(y, clf.predict(X), y_test, clf.predict(X_test))

In [42]:
def plot_roc_auc(y_train, y_train_pred, y_test, y_test_pred, title):
    
    fpr, tpr, _ = roc_curve(y_train, y_train_pred)
    auc = roc_auc_score(y_train, y_train_pred)

    plt.plot(fpr, tpr, color='blue',
                 label="Навчальна вибірка, AUC={:.3f}".format(auc), linestyle='-')


    fpr, tpr, _ = roc_curve(y_test,  y_test_pred)
    auc = roc_auc_score(y_test, y_test_pred)

    plt.plot(fpr, tpr, color='black',
                 label="Тестова вибірка, AUC={:.3f}".format(auc), linestyle='--')



    plt.plot([0,1], [0,1], color='gray', linestyle='--')

    plt.xticks(np.arange(0.0, 1.1, step=0.1))
    plt.xlabel("Flase Positive Rate", fontsize=15)

    plt.yticks(np.arange(0.0, 1.1, step=0.1))
    plt.ylabel("True Positive Rate", fontsize=15)

    plt.title(title, fontweight='bold', fontsize=15)
    plt.legend(prop={'size':13}, loc='lower right')

    plt.show()

In [ ]:
plot_roc_auc(y_train, pred_lgb, y_test, pred_lgb_test, "LightGBM")